# Final HiCoTiNe implementation on MNIST dataset.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score
from keras import backend as K
import cv2

In [ ]:
SUB_REGION_SCALE = 14
MIN_SUB_REGION_SCALE = 12
MAX_SUB_REGION_SCALE = 16
NUM_HICO_LAYER = 5
NUM_TN = [10, 8, 6, 4, 2]
NUM_CONNECTION = [0, 5, 4, 3, 2]
ALPHA = 0.1

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [ ]:
#load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
#Generate cropped train image
X_train_cropped_list = []
coordinate_list = []
scale_list = []
for i in range(NUM_TN[0]):
  X_train_cropped = []
  sub_region_scale = random.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
  x = random.randint(0, X_train.shape[1] - sub_region_scale)
  y = random.randint(0, X_train.shape[1] - sub_region_scale)
  coordinate = (x, y)
  for j in range(X_train.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_row.append(X_train[j][row][column])
      image_cropped.append(image_cropped_row)
    X_train_cropped.append(image_cropped)
  
  X_train_cropped = np.array(X_train_cropped)
  X_train_cropped_list.append(X_train_cropped)
  coordinate_list.append(coordinate)
  scale_list.append(sub_region_scale)

In [ ]:
#Generate cropped test image
X_test_cropped_list = []
for i in range(NUM_TN[0]):
  X_test_cropped = []
  sub_region_scale = scale_list[i]
  x = coordinate_list[i][0]
  y = coordinate_list[i][1]
  for j in range(X_test.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_row.append(X_test[j][row][column])
      image_cropped.append(image_cropped_row)
    X_test_cropped.append(image_cropped)

  X_test_cropped = np.array(X_test_cropped)
  X_test_cropped_list.append(X_test_cropped)

In [ ]:
#rescaling dataset
for i in range(NUM_TN[0]):
  temp = []
  for j in range (X_train.shape[0]):
    temp.append(cv2.resize(X_train_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_train_cropped_list[i] = temp

  temp = []
  for j in range(X_test.shape[0]):
    temp.append(cv2.resize(X_test_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_test_cropped_list[i] = temp

In [ ]:
#normalizing dataset
for i in range(NUM_TN[0]):
  X_train_cropped_list[i] = X_train_cropped_list[i]/255
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [ ]:
#transform to vector
for i in range(NUM_TN[0]):
  X_train_cropped_list[i] = X_train_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))

In [ ]:
#one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

## HiCo Layer 1

In [ ]:
#build ANN model
ensemble = []
for i in range(NUM_TN[0]):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [ ]:
#train model
history = []
for i in range(NUM_TN[0]):
  print('Model %d' %i)
  hist = ensemble[i][0].fit(X_train_cropped_list[i], y_train_one_hot, validation_data=(X_test_cropped_list[i], y_test_one_hot), epochs=5, batch_size=128)
  history.append(hist)

Model 0
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 1.0545 - accuracy: 0.6803 - val_loss: 0.4388 - val_accuracy: 0.8639
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.4093 - accuracy: 0.8744 - val_loss: 0.3627 - val_accuracy: 0.8860
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.3405 - accuracy: 0.8944 - val_loss: 0.3346 - val_accuracy: 0.8916
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.3062 - accuracy: 0.9026 - val_loss: 0.3032 - val_accuracy: 0.9022
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.2840 - accuracy: 0.9104 - val_loss: 0.2989 - val_accuracy: 0.9016
Model 1
Epoch 1/5
469/469 [==============================] - 2s 2ms/step - loss: 0.9614 - accuracy: 0.7084 - val_loss: 0.3424 - val_accuracy: 0.8931
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.3342 - accuracy: 0.8932 - val_loss: 0.2800 - val_accuracy: 0.90

In [ ]:
hico_layers = []
hico_layers.append(ensemble)

## HiCo Layer 2+

In [ ]:
# Function to get input of layer i-1
def get_previous_layer_input(hico_layers, layer, tn, train_image):
  input = []
    
  if layer == 0:
    previous_layer_input = train_image[tn[1]]
    return previous_layer_input

  elif layer > 0:
    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[1].output))
      input.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))
      
    input = np.array(input)
    input = np.concatenate(input, axis=1)
    previous_layer_input = input
    return previous_layer_input

In [ ]:
for i in range(1, NUM_HICO_LAYER):
  print('Layer %d' %i)
  ensemble = []
  X_train_input_list = []
  X_test_input_list = []

  for j in range(NUM_TN[i]):
    # Build model of HiCo layer i
    connection = tuple(random.sample(range(len(hico_layers[i-1])), k=NUM_CONNECTION[i]))
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=64*NUM_CONNECTION[i]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append((model, connection))
  hico_layers.append(ensemble)
  print('HICO LENGTH')
  print(len(hico_layers))

  for j in range(NUM_TN[i]):
    # Get train hidden representation from HiCo layer i-1
    X_train_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_train_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list)))
    X_train_input = np.array(X_train_input)
    X_train_input = np.concatenate(X_train_input, axis=1)
    X_train_input_list.append(X_train_input)

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    X_test_input_list.append(X_test_input)

  #train model of HiCo layer i
  for j in range(NUM_TN[i]):
    print('Model %d' %j)
    hico_layers[i][j][0].fit(X_train_input_list[j], y_train_one_hot, validation_data=(X_test_input_list[j], y_test_one_hot), epochs=5, batch_size=128)

Layer 1
HICO LENGTH
2
Model 0
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.3769 - accuracy: 0.8943 - val_loss: 0.0908 - val_accuracy: 0.9718
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0769 - accuracy: 0.9762 - val_loss: 0.0791 - val_accuracy: 0.9748
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0676 - accuracy: 0.9790 - val_loss: 0.0748 - val_accuracy: 0.9757
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0594 - accuracy: 0.9803 - val_loss: 0.0815 - val_accuracy: 0.9739
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0587 - accuracy: 0.9816 - val_loss: 0.0738 - val_accuracy: 0.9775
Model 1
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.4861 - accuracy: 0.8687 - val_loss: 0.1055 - val_accuracy: 0.9675
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0813 - accuracy: 0.9749 - val_loss: 0.088

## Model Evaluation

In [ ]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = hico_layers[0][j][0].predict(X_test_cropped_list[j])
  y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[0] / max(NUM_TN)))
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    y_pred = hico_layers[i][j][0].predict(X_test_input)
    y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[i] / max(NUM_TN)))
    y_pred_list.append(y_pred)


# HiCo voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0, 2))
y_pred_list = np.argmax(np.sum(y_pred_list, axis=1), axis=1)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.9847